In [173]:
from imp import reload
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys

sys.path.append("..")

import d2l_pytorch.d2l as d2l

reload(d2l)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [142]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics(
  "F:\python_code\DL\Datasets\JayChouLyrics\jaychou_lyrics.txt"
)

In [143]:
def one_hot(x, n_class, dtype=torch.float32):
  x = x.long()
  res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
  res.scatter_(1, x.view(-1, 1), 1)
  return res


In [144]:
x = torch.tensor([0, 2])
x.long()
# one_hot(x, vocab_size)
F.one_hot(x, vocab_size)


tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]])

In [145]:
X = torch.arange(10).view(2, 5)
inputs = d2l.to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)


5 torch.Size([2, 1028])


In [146]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print("will use", device)


will use cuda


In [147]:
def get_params():
  def _one(shape):
    ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
    return torch.nn.Parameter(ts, requires_grad=True)

  W_xh = _one((num_inputs, num_hiddens))
  W_hh = _one((num_hiddens, num_hiddens))
  b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
  W_hq = _one((num_hiddens, num_outputs))
  b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
  return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])


In [148]:
# 定义模型
def init_rnn_state(batch_size, num_hiddens, device):
  return (torch.zeros((batch_size, num_hiddens), device=device),)


In [149]:
def rnn(inputs, state, params):
  W_xh, W_hh, b_h, W_hq, b_q = params
  H, = state
  outputs = []
  for X in inputs:
    H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
    Y = torch.matmul(H, W_hq) + b_q
    outputs.append(Y)

  return outputs, (H,)


In [150]:
from d2l_pytorch.d2l import to_onehot


state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)


5 torch.Size([2, 1028]) torch.Size([2, 256])


In [151]:
# 定义预测函数
def predict_rnn(
  prefix,
  num_chars,
  rnn,
  params,
  init_rnn_state,
  num_hiddens,
  vocab_size,
  device,
  idx_to_char,
  char_to_idx,
):
  state = init_rnn_state(1, num_hiddens, device)
  output = [char_to_idx[prefix[0]]]

  for t in range(num_chars + len(prefix) - 1):
    X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
    (Y, state) = rnn(X, state, params)

    if t < len(prefix) - 1:
      output.append(char_to_idx[prefix[t + 1]])
    else:
      output.append(int(Y[0].argmax(dim=1).item()))

  return "".join([idx_to_char[i] for i in output])


In [152]:
predict_rnn(
  "分开", 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, device, idx_to_char, char_to_idx
)

'分开壶毫刀掩侬晚擅传否甜'

In [153]:
def grad_clipping(params, theta, device):
  norm = torch.tensor([0.0], device=device)
  for param in params:
    norm += (param.grad.data**2).sum()

  norm = norm.sqrt().item()
  if norm > theta:
    for param in params:
      param.grad.data *= (theta / norm)


In [154]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ["分开","不分开"]

In [156]:
from d2l_pytorch.d2l import train_and_predict_rnn


train_and_predict_rnn(
  rnn,
  get_params,
  init_rnn_state,
  num_hiddens,
  vocab_size,
  device,
  corpus_indices,
  idx_to_char,
  char_to_idx,
  True,
  num_epochs,
  num_steps,
  lr,
  clipping_theta,
  batch_size,
  pred_period,
  pred_len,
  prefixes,
)


epoch 50, perplexity 77.298071, time 0.46 sec
 - 分开 我不要再生你的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯
 - 不分开我
我有我有女人 一场哈兮 在小村外 在小村外 我有一场 在小村外 我有一场 在小村外 我有一场 在
epoch 100, perplexity 9.887331, time 0.39 sec
 - 分开
我已想这样牵着你的手不放开
爱可不能够永远单纯没有悲害
你 靠着我的肩膀
你 想不胸口阳车
我说想
 - 不分开吗
我想想你 你着我 别我 我 我不多
我给就这样牵
后知后觉
又已了离 我 已带球
我给能这样活

epoch 150, perplexity 2.822328, time 0.45 sec
 - 分开我想妈
就小我的让我 一檐是剧
我一道的生活 我爱你 你爱我
不要了枪
喜堡好双截棍 哼哼哈兮
快使
 - 不分开期
我想你和 你打我妈
这样了我已始
没有安我对地的暴
别言准备重袭
我该念起国小的课桌就
用铅一只
epoch 200, perplexity 1.566349, time 0.36 sec
 - 分开的狗萨 问铁变风 全背它纵
恨底对中 象一场梦 不敢去碰 你去操
连我该轻重 有慢的衷走
能失什么映
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜
她养的黑猫笑起来像哭 啦啦啦呜 
你的回旧简问 太彻会
让我连恨
epoch 250, perplexity 1.319662, time 0.50 sec
 - 分开不 像堡去回忆
你在那  你 得下出
我手眼的叹息
已风安著风
一天就到落里来到
陷头让危险边缘Ba
 - 不分开期
我叫你爸 你打我妈
这样对吗干嘛这样
何必让酒牵鼻子走
瞎 说了它比谁
难领 我爸 我不能再想你


In [175]:
# 简洁实现

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics("F:\python_code\DL\Datasets\JayChouLyrics\jaychou_lyrics.txt")

num_hiddens = 256
rnn_layer = nn.RNN(input_size = vocab_size, hidden_size=num_hiddens)
num_steps = 35
batch_size = 2
state = None

X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [174]:
class RNNModel(nn.Module):
  def __init__(self, rnn_layer, vocab_size):
    super(RNNModel, self).__init__()
    self.rnn = rnn_layer
    self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
    self.vocab_size = vocab_size
    self.dense = nn.Linear(self.hidden_size, vocab_size)
    self.state = None

  def forward(self, inputs, state):
    X = d2l.to_onehot(inputs, self.vocab_size)
    Y, self.state = self.rnn(torch.stack(X), state)
    output = self.dense(Y.view(-1, Y.shape[-1]))
    return output, self.state


In [176]:
model = RNNModel(rnn_layer, vocab_size).to(device)
d2l.predict_rnn_pytorch("在", 10, model, vocab_size, device, idx_to_char, char_to_idx)

'在哀容战战战酿战酿战酿'

In [177]:
from d2l_pytorch.d2l import train_and_predict_rnn_pytorch


num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ["分开", "不分开"]
train_and_predict_rnn_pytorch(
  model,
  num_hiddens,
  vocab_size,
  device,
  corpus_indices,
  idx_to_char,
  char_to_idx,
  num_epochs,
  num_steps,
  lr,
  clipping_theta,
  batch_size,
  pred_period,
  pred_len,
  prefixes,
)


 epoch 50, perplexity 9.016533 , time 0.26 sec 
 - 分开 我不了口想

说在那里 哼哼哈兮
快使用双截棍 哼哼哈兮
快使用双截棍 哼哼哈兮
快使用双截棍 哼
 - 不分开不能再想著你 是我的的爱边河

想你的手在着开的可爱
人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的
 epoch 100, perplexity 1.268239 , time 0.10 sec 
 - 分开 我不多不力
你说你直在我来不知道你前一切落我不能
想通 却又再考倒我
说散 你想很久了吧?
败给你
 - 不分开不了就想的你
快使用双截棍 哼哼哈兮
快使用双截棍 哼哼哈兮
如果我有轻功 飞檐走壁
为人耿直不屈 
 epoch 150, perplexity 1.068386 , time 0.11 sec 
 - 分开 我不多不除
你爱就直是你
想要你 陪我久了太多
你说完美就离
一直和汉堡 
想要你的微笑每天都能看
 - 不分开不了口不开
武它
是家庭的没有久
我不懂 你你在 别离的太多
我想是你是雨是悲剧
是说完美演出 一场
 epoch 200, perplexity 1.034696 , time 0.11 sec 
 - 分开 我不了口让

道过我不想我想要你想你
一只会说好语举
如果真到掉的让我想和你这样
快样的回忆对 我
 - 不分开你了它一口好生
快我它的回头 
路完美主义 太彻底
分手的话像语言暴力
我已无能为力再提起 决定中断
 epoch 250, perplexity 1.023370 , time 0.09 sec 
 - 分开 我不了口让

道过我是那我想要看到我

想要和你又会听时样
是说为没有你在 我不多
我天都没有你在
 - 不分开你听它一开始生活
我感的是我场悲剧
我可以让生命就这样毫无意义
或许在最后能听到你一句
轻轻的叹息 
